# Finding and using anchor points

In this notebook, we show how to find anchor points based on your training set and how to use them to estimate the performance of new models in the test set.

## Preparing data

Loading packages

In [57]:
import numpy as np
import pickle
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import pairwise_distances
from irt import *
from utils import *

random_state = 42

The leaderboard dataset we will use is composed by six scenarios (sub-datasets):
1. TruthfulQA
1. GSM8K
1. Winogrande
1. ARC
1. HellaSwag
1. MMLU

MMLU is further divided into sub-scenarios (e.g., abstract algebra, anatomy, etc). Let's check scenarios and sub-scenarios:

In [58]:
scenarios

{'harness_truthfulqa_mc_0': ['harness_truthfulqa_mc_0'],
 'gsm8k': ['harness_gsm8k_5'],
 'winogrande': ['harness_winogrande_5'],
 'arc': ['harness_arc_challenge_25'],
 'hellaswag': ['harness_hellaswag_10'],
 'mmlu': ['harness_hendrycksTest_abstract_algebra_5',
  'harness_hendrycksTest_anatomy_5',
  'harness_hendrycksTest_astronomy_5',
  'harness_hendrycksTest_business_ethics_5',
  'harness_hendrycksTest_clinical_knowledge_5',
  'harness_hendrycksTest_college_biology_5',
  'harness_hendrycksTest_college_chemistry_5',
  'harness_hendrycksTest_college_computer_science_5',
  'harness_hendrycksTest_college_mathematics_5',
  'harness_hendrycksTest_college_medicine_5',
  'harness_hendrycksTest_college_physics_5',
  'harness_hendrycksTest_computer_security_5',
  'harness_hendrycksTest_conceptual_physics_5',
  'harness_hendrycksTest_econometrics_5',
  'harness_hendrycksTest_electrical_engineering_5',
  'harness_hendrycksTest_elementary_mathematics_5',
  'harness_hendrycksTest_formal_logic_5',
 

In [59]:
SELECTED_SCENARIOS = ['gsm8k', 'arc', 'hellaswag', 'harness_truthfulqa_mc_0']

# select gsm8k, arc, hellaswag
lb_scenarios = {'lb': []}
for scenario in scenarios.keys():
    if scenario in SELECTED_SCENARIOS:
        lb_scenarios['lb'].append(*scenarios[scenario])

Loading leaderboard data:

In [60]:
#with open('data/lb.pickle', 'rb') as handle:
#    data = pickle.load(handle)
with open('data/lb_scenarios.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [61]:
for s in data['data'].keys():
    print(s)
    print('Number of nan values in data:', np.sum(np.isnan(data['data'][s]['correctness'])))

harness_hellaswag_10
Number of nan values in data: 0
harness_truthfulqa_mc_0
Number of nan values in data: 104
harness_arc_challenge_25
Number of nan values in data: 0
harness_gsm8k_5
Number of nan values in data: 0


In this dataset, we have data from 395 models. Let's see the names of some of them below

In [62]:
lb_scenarios

{'lb': ['harness_truthfulqa_mc_0',
  'harness_gsm8k_5',
  'harness_arc_challenge_25',
  'harness_hellaswag_10']}

In [63]:
len(data['models']),data['models'][:10]

(1,
 [['open-llm-leaderboard/details_moreh__MoMo-70B-lora-1.8.6-DPO',
   'open-llm-leaderboard/details_cloudyu__Yi-34Bx3-MoE-90B',
   'open-llm-leaderboard/details_Weyaxi__Helion-4x34B',
   'open-llm-leaderboard/details_Weyaxi__Bagel-Hermes-34B-Slerp',
   'open-llm-leaderboard/details_Weyaxi__Bagel-Hermes-2x34b',
   'open-llm-leaderboard/details_nfaheem__Marcoroni-7b-DPO-Merge',
   'open-llm-leaderboard/details_jondurbin__bagel-dpo-34b-v0.2',
   'open-llm-leaderboard/details_udkai__Turdus',
   'open-llm-leaderboard/details_gagan3012__MetaModel_moe',
   'open-llm-leaderboard/details_jeonsworld__CarbonVillain-en-10.7B-v3',
   'open-llm-leaderboard/details_TomGrc__FusionNet',
   'open-llm-leaderboard/details_kekmodel__StopCarbon-10.7B-v6',
   'open-llm-leaderboard/details_jeonsworld__CarbonVillain-en-10.7B-v1',
   'open-llm-leaderboard/details_Weyaxi__SauerkrautLM-UNA-SOLAR-Instruct',
   'open-llm-leaderboard/details_VAGOsolutions__SauerkrautLM-SOLAR-Instruct',
   'open-llm-leaderboard/de

Below, we will process the data so all correctness scores (for all scenarios) are stored in $Y$. The dictionaries `scenarios_position` and `subscenarios_position` give the position of scenarios/subscenarios correctness scores in $Y$.

In [64]:
scenarios_position, subscenarios_position = prepare_data(lb_scenarios, data)
Y = create_responses(lb_scenarios, data)
Y.shape

(393, 13350)

For example, below you can see the scores for MMLU:

In [65]:
Y[:,scenarios_position['lb']], Y[:,scenarios_position['lb']].shape

(array([[0.99999999, 0.99994655, 1.        , ..., 1.        , 0.        ,
         1.        ],
        [0.99984026, 0.99999999, 0.99908489, ..., 1.        , 1.        ,
         1.        ],
        [0.99937792, 0.99999997, 0.99732544, ..., 1.        , 1.        ,
         1.        ],
        ...,
        [0.59677787, 0.99801392, 0.20934594, ..., 0.        , 0.        ,
         0.        ],
        [0.67288482, 0.99814252, 0.3865014 , ..., 0.        , 1.        ,
         1.        ],
        [0.42345796, 0.99926741, 0.82057698, ..., 0.        , 0.        ,
         0.        ]]),
 (393, 13350))

In [66]:
# fill nan values with 0
Y[np.isnan(Y)] = 0

# print stats of Y
print('Y stats:')
print('min:', np.min(Y))
print('max:', np.max(Y))
print('mean:', np.mean(Y))
print('std:', np.std(Y))

Y stats:
min: 0.0
max: 1.0000000000000002
mean: 0.7231160348021274
std: 0.443520043003996


For scenarios that have multiple subscenarios, it is usually the case that we want to give equal importance to individual subscenarios when computing the aggregated performance in that scenario. This is equivalent to using a weighted average when computing the aggregated performance. We will create `balance_weights`, a vector of weights to help us compute those weighted averages. These weights will be different than one only for MMLU, which is the only scenario with multiple subscenarios.

In [67]:
balance_weights = np.ones(Y.shape[1])

selected_scenarios = lb_scenarios['lb']

N = len(scenarios_position['lb'])
n_sub = len(selected_scenarios)
for sub in selected_scenarios:
    n_i = len(subscenarios_position['lb'][sub])
    balance_weights[subscenarios_position['lb'][sub]] = N/(n_sub*n_i)  

We can see below that first averaging within subscenarios and then computing a simple average is equivalent to using a weighted average from the beginning:

In [68]:
accs1 = np.mean([Y[:,subscenarios_position['lb'][sub]].mean(axis=1) for sub in lb_scenarios['lb']], axis=0)
accs2 = (balance_weights*Y)[:,scenarios_position['lb']].mean(axis=1)

np.abs(accs1 - accs2).mean()

1.1356762293373007e-14

## Getting and using anchor points

Let's split the data in train and test (recent models are placed in the test set):

In [69]:
Y_test = Y[:100]
Y_train = Y[100:]

In [70]:
(balance_weights*Y_train)[:,scenarios_position['lb']].mean(axis=1).max()

0.6547273254796847

The variable `number_item` gives the number of anchor points we want to find in each scenario:

The variable `clustering` specified how the clusting is run. If `clustering="correct."`, then correctness is used. On the other hand, if `clustering="irt"`, then the IRT embeddings for examples are used.

Computing anchor points and their weights for each scenario:

In [71]:
def compute_anchor_points(clustering, lb_scenarios, scenarios_position, balance_weights, Y_train, number_item, random_state):
    anchor_points = {}
    anchor_weights = {}

    for scenario in lb_scenarios.keys():

        if clustering=='correct.':
            X = Y_train[:,scenarios_position[scenario]].T
        elif clustering=='irt':
            A, B, _ = load_irt_parameters('data/irt_model/')
            X = np.vstack((A.squeeze(), B.squeeze().reshape((1,-1)))).T
            X = X[scenarios_position[scenario]]
        else:
            raise NotImplementedError 
            
        #Normalizing balance_weights, so their sum is one within each scenario
        norm_balance_weights = balance_weights[scenarios_position[scenario]]
        norm_balance_weights /= norm_balance_weights.sum()

        # Fitting the KMeans model
        kmeans = KMeans(n_clusters=number_item, n_init="auto", random_state=random_state)
        kmeans.fit(X, sample_weight=norm_balance_weights)

        # Calculating anchor points
        anchor_points[scenario] = pairwise_distances(kmeans.cluster_centers_, X, metric='euclidean').argmin(axis=1)

        # Calculating anchor weights
        anchor_weights[scenario] = np.array([np.sum(norm_balance_weights[kmeans.labels_==c]) for c in range(number_item)])

    return anchor_points, anchor_weights

Saving

In [72]:
clustering = 'irt' # 'correct.' or 'irt'

for anchor_num in [10, 15, 20, 30, 50]:
    anchor_points, anchor_weights = compute_anchor_points(clustering, lb_scenarios, scenarios_position, balance_weights, Y_train, anchor_num, random_state)
    anchor = {'anchor_points':anchor_points,
            'anchor_weights':anchor_weights}

    # save to tinybenchmark_lb file, putting 'seen_examples', 'examples_weights', 'irt_parameters', 'scenarios_position', 'subscenarios_position', 'optimal_lambdas'
    tinybenchmark_lb = {'seen_examples':anchor_points,
                        'examples_weights':anchor_weights,
                        'scenarios_position':scenarios_position,
                        'subscenarios_position':subscenarios_position,
                        }

    with open(f'data/lb_anchor_{anchor_num}.pickle', 'wb') as handle:
        pickle.dump(tinybenchmark_lb, handle, protocol=pickle.HIGHEST_PROTOCOL)

Checking results

In [73]:
tinybenchmark_lb['seen_examples']

{'lb': array([  865,  7421,   413,  2193,   449,  1290,   511,   227,   698,
         7286,  1277,  7663,   746,  5910, 12947,  2866, 12972,  5336,
         5912,   470,  1766,  1193,   615,  6103,   124,  1622,  1068,
          831,  1167,  3164,  2968,  3042,  9778,  2678, 10076,  6063,
         1863,    55, 10614,  5644,  3361,  4207,  3833,  8169, 10490,
          247,  1680,  1762,   484,  2472])}

Using anchor points to estimate performance in the test set and reporting the average prediction error

In [75]:
for scenario in lb_scenarios.keys():
    Y_anchor = Y_test[:,scenarios_position[scenario]][:,anchor_points[scenario]]
    Y_hat = (Y_anchor*anchor_weights[scenario]).sum(axis=1)
    Y_true = (balance_weights*Y_test)[:,scenarios_position[scenario]].mean(axis=1)

    print(f"scenario: {scenario}, avg. error: {np.abs(Y_hat-Y_true).mean():.3f}")

scenario: lb, avg. error: 0.031
